<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/Training_ALIGNN_model_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ALIGNN](https://github.com/usnistgov/alignn) example

In [1]:
import os
# Clone ALIGNN repo
if not os.path.exists('alignn'):
  !git clone https://github.com/usnistgov/alignn.git

os.chdir('alignn')
# Install
!python setup.py develop

!pip install dgl-cu111 # Colab has cuda 11.1

Cloning into 'alignn'...
remote: Enumerating objects: 2003, done.
remote: Counting objects: 100% (2003/2003), done.
remote: Compressing objects: 100% (1020/1020), done.
remote: Total 2003 (delta 1275), reused 1571 (delta 924), pack-reused 0
Receiving objects: 100% (2003/2003), 1.04 MiB | 8.96 MiB/s, done.
Resolving deltas: 100% (1275/1275), done.
running develop
running egg_info
creating alignn.egg-info
writing alignn.egg-info/PKG-INFO
writing dependency_links to alignn.egg-info/dependency_links.txt
writing entry points to alignn.egg-info/entry_points.txt
writing requirements to alignn.egg-info/requires.txt
writing top-level names to alignn.egg-info/top_level.txt
writing manifest file 'alignn.egg-info/SOURCES.txt'
adding license file 'LICENSE.rst'
writing manifest file 'alignn.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/alignn.egg-link (link to .)
Adding alignn 2021.11.13 to easy-install.pth file
Installing train_folder=alignn.script.train_fo

Example folder with id_prop.csv and 'POSCAR files.'

In [2]:
!ls "alignn/examples/sample_data"

config_example.json	  POSCAR-JVASP-64045.vasp  POSCAR-JVASP-86097.vasp
id_prop.csv		  POSCAR-JVASP-64240.vasp  POSCAR-JVASP-86205.vasp
POSCAR-JVASP-107772.vasp  POSCAR-JVASP-64377.vasp  POSCAR-JVASP-86436.vasp
POSCAR-JVASP-10.vasp	  POSCAR-JVASP-64584.vasp  POSCAR-JVASP-86726.vasp
POSCAR-JVASP-13526.vasp   POSCAR-JVASP-64664.vasp  POSCAR-JVASP-86968.vasp
POSCAR-JVASP-1372.vasp	  POSCAR-JVASP-64719.vasp  POSCAR-JVASP-89025.vasp
POSCAR-JVASP-14014.vasp   POSCAR-JVASP-64906.vasp  POSCAR-JVASP-89265.vasp
POSCAR-JVASP-14441.vasp   POSCAR-JVASP-65062.vasp  POSCAR-JVASP-90228.vasp
POSCAR-JVASP-14873.vasp   POSCAR-JVASP-65101.vasp  POSCAR-JVASP-90532.vasp
POSCAR-JVASP-15345.vasp   POSCAR-JVASP-655.vasp    POSCAR-JVASP-90856.vasp
POSCAR-JVASP-1996.vasp	  POSCAR-JVASP-676.vasp    POSCAR-JVASP-97378.vasp
POSCAR-JVASP-21210.vasp   POSCAR-JVASP-76308.vasp  POSCAR-JVASP-97499.vasp
POSCAR-JVASP-22556.vasp   POSCAR-JVASP-76309.vasp  POSCAR-JVASP-97570.vasp
POSCAR-JVASP-27901.vasp   POSCAR-JVASP-76312.

# 50 materials and their bandgap data generated with the script [generate_sample_data_reg.py](https://github.com/usnistgov/alignn/blob/main/alignn/examples/sample_data/scripts/generate_sample_data_reg.py)

# Train a model for 3 epochs and batch size of 2. Other parameters are provided in `config_example.json` file.

In [3]:
import time
t1=time.time()
!python alignn/scripts/train_folder.py --root_dir "alignn/examples/sample_data" --epochs 3 --config "alignn/examples/sample_data/config_example.json" --output_dir=temp
t2=time.time()
print ('Time in s',t2-t1)

DGL backend not selected or invalid.  Assuming PyTorch for now.
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Using backend: pytorch
MAX val: 6.149
MIN val: 0.0
MAD: 1.0520696
Baseline MAE: 0.7102749999999998
data range 6.149 0.0
  0% 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/jarvis_tools-2021.10.3-py3.7.egg/jarvis/core/graphs.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  r = torch.tensor(r).type(torch.get_default_dtype())
100% 40/40 [00:01<00:00, 22.37it/s]
Setting it to max atomic number available here, 103
Setting it to max atomic number available here, 103
Setting it to max atomic number avail

Use pretrained models such as models trained on JARVIS-DFT, QM9, Materials project, hMOF etc. databases

In [4]:
!python alignn/pretrained.py -h

Using backend: pytorch
usage: pretrained.py [-h] [--model_name MODEL_NAME]
                     [--file_format FILE_FORMAT] [--file_path FILE_PATH]
                     [--cutoff CUTOFF]

Atomistic Line Graph Neural Network Pretrained Models

optional arguments:
  -h, --help            show this help message and exit
  --model_name MODEL_NAME
                        Choose a model from these 38
                        models:jv_formation_energy_peratom_alignn,
                        jv_optb88vdw_total_energy_alignn,
                        jv_optb88vdw_bandgap_alignn, jv_mbj_bandgap_alignn,
                        jv_spillage_alignn, jv_slme_alignn,
                        jv_bulk_modulus_kv_alignn, jv_shear_modulus_gv_alignn,
                        jv_n-Seebeck_alignn, jv_n-powerfact_alignn,
                        jv_magmom_oszicar_alignn,
                        jv_kpoint_length_unit_alignn, jv_avg_elec_mass_alignn,
                        jv_avg_hole_mass_alignn, jv_epsx_alignn,


In [6]:
!python alignn/pretrained.py --model_name jv_formation_energy_peratom_alignn --file_format poscar --file_path alignn/examples/sample_data/POSCAR-JVASP-10.vasp

Using backend: pytorch
100% 47.5M/47.5M [00:03<00:00, 14.6MiB/s]
name='alignn' alignn_layers=4 gcn_layers=4 atom_input_features=92 edge_input_features=80 triplet_input_features=40 embedding_features=64 hidden_features=256 output_features=1 link='identity' zero_inflated=False classification=False
/usr/local/lib/python3.7/dist-packages/jarvis_tools-2021.10.3-py3.7.egg/jarvis/core/graphs.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  r = torch.tensor(r).type(torch.get_default_dtype())
Predicted value: jv_formation_energy_peratom_alignn alignn/examples/sample_data/POSCAR-JVASP-10.vasp [-0.7033944725990295]
